![](https://i.imgur.com/A3Cxp7P.png)

----

- [Some analysis of Goals...](#Some-analysis-of-Goals...)
    - [Goals scored vs conceded per team](#Goals-scored-vs-conceded-per-team)
    - [Goal period](#Goal-period)
    - [Shirt number matters...](#Shirt-number-matters...)
    
    
- [Some team analysis...](#Some-team-analysis...)
    - [What was the best stage native/foreign coaches were able to take their team to?](#What-was-the-best-stage-native/foreign-coaches-were-able-to-take-their-team-to?)
    
    
- [Some analysis on players...](#Some-analysis-on-players...)
    - [Goalkeepers are biggg!!!](#Goalkeepers-are-biggg!!!)
    - [Some age analysis](#Some-age-analysis)
    - [None of the teams with average age above 28 could make it past Quarterfinals...](#None-of-the-teams-with-average-age-above-28-could-make-it-past-Quarterfinals...)
    
    
- [Some match attendance analysis...](#Some-match-attendance-analysis...)
    - [Why so low attendance in quarters? Maybe bcoz of no Portugal, Spain and Argentina!](#Why-so-low-attendance-in-quarters?-Maybe-bcoz-of-no-Portugal,-Spain-and-Argentina!)
    
    
- [Some analysis on team tactics...](#Some-analysis-on-team-tactics...)
    - [4-2-3-1 seems to be everyone's favorite. Does it guarantee a win?](#4-2-3-1-seems-to-be-everyone's-favorite.-Does-it-guarantee-a-win?)
    - [How does 4-2-3-1 tactic fares at big stage?](#How-does-4-2-3-1-tactic-fares-at-big-stage?)
    
    
- [Some analysis on ball possession...](#Some-analysis-on-ball-possession...)
    - [In 4/6 matches, France had lesser ball possession than the opponent..](#In-4/6-matches,-France-had-lesser-ball-possession-than-the-opponent..)
    
----

In [ ]:
import pandas as pd
%matplotlib inline

In [ ]:
teams_df = pd.read_csv("data/teams.csv", index_col="IdTeam")

In [ ]:
teams_df.head()

In [ ]:
players_df = pd.read_csv("data/players.csv", parse_dates=['BirthDate'], index_col="IdPlayer")

In [ ]:
players_df.head()

In [ ]:
matches_df = pd.read_csv("data/matches.csv", parse_dates=['Date'], index_col="IdMatch")

In [ ]:
matches_df.head()

In [ ]:
goals_df = pd.read_csv("data/goals.csv")

In [ ]:
goals_df.head()

## Some analysis of Goals...
![](https://i.imgflip.com/219r12.jpg)

### Goals scored vs conceded per team

In [ ]:
goals_scored_per_team = goals_df['TeamName'].value_counts()

In [ ]:
goals_conceded_per_team = goals_df['OppositionTeamName'].value_counts()

In [ ]:
goals_scored_per_team.get('Croatia')

In [ ]:
teams_df['GoalsScored'] = teams_df['TeamName'].apply(goals_scored_per_team.get)

In [ ]:
teams_df['GoalsConceded'] = teams_df['TeamName'].apply(goals_conceded_per_team.get)

In [ ]:
# goals scored vs conceded
teams_df.plot.bar(x='TeamName', y=['GoalsScored', 'GoalsConceded'], figsize=(15,6))

In [ ]:
# Which teams conceded more goals than they scored?
teams_df.query('GoalsScored < GoalsConceded')

### Goal period

In [ ]:
def set_goal_period(minute):
    total = eval(minute.replace('\'', ''))
    if "45'+" in minute or total<=45:
        return "FirstHalf"
    elif  "90'+" in minute or 45<total<=90:
        return "SecondHalf"
    elif 90<total<=120:
        return "ExtraTime"
    else:
        return "PenaltyShootout"

In [ ]:
goals_df['Period'] = goals_df['Minute'].apply(set_goal_period)

In [ ]:
goals_df['Period'].value_counts().plot.pie(autopct='%1.1f%%')

### Shirt number matters...

In [ ]:
# 10 vs 7
goals_df['PlayerShirtNumber'].value_counts()

In [ ]:
# players with shirt no. 10 who scored goals 
goals_df.query("PlayerShirtNumber == 10")['PlayerName'].value_counts()

## Some team analysis...
![](https://michaelsloredotcom.files.wordpress.com/2017/12/545399686-france-team-members-celebrate-after-beating-germany-2-0-crop-promo-xlarge2-e1512248877901.jpg?w=845&h=450&crop=1)

In [ ]:
# Best stage teams made?
best_stage_map = {}
for i in range(len(matches_df.index)):
    match = matches_df.iloc[i]
    best_stage_map[match['HomeTeamName']] = match['Stage']
    best_stage_map[match['AwayTeamName']] = match['Stage']

In [ ]:
best_stage_map

In [ ]:
teams_df['BestStage'] = teams_df['TeamName'].apply(best_stage_map.get)

### What was the best stage native/foreign coaches were able to take their team to?

In [ ]:
teams_df.query('TeamName != CoachCountry')['BestStage'].value_counts().plot.pie(autopct='%1.1f%%')

In [ ]:
teams_df.query('TeamName == CoachCountry')['BestStage'].value_counts().plot.pie(autopct='%1.1f%%')

## Some analysis on players...
![](http://n.sinaimg.cn/sports/transform/224/w615h409/20180707/hZYi-hexfcvm1472143.jpg)

In [ ]:
players_df['Height'].plot.hist()

In [ ]:
# Who is tallest player?
players_df.loc[players_df['Height'].idxmax()]

In [ ]:
# who is the tiniest?
players_df.loc[players_df['Height'].idxmin()]

In [ ]:
# Player position breakdown
players_df.Position.value_counts()

In [ ]:
def get_position_color(position):
    color_map = {
        "Defender": 'y',
        "Midfielder": 'b',
        "Forward": 'g',
        "Goalkeeper": 'r'
    }
    return color_map[position]

In [ ]:
color_map = players_df['Position'].apply(get_position_color)

In [ ]:
players_df.plot.scatter(x='Height', y='Weight', c=color_map)

In [ ]:
players_df.groupby('Position')[['Height', 'Weight']].mean()

### Goalkeepers are biggg!!!
![](https://i.imgur.com/v81dlTt.jpg)

### Some age analysis

![](https://t.resfu.com/media/img_news/montage-of-essam-el-hadary-and-kylian-mbappe--besoccer.png?size=776x&q=60)

In [ ]:
from datetime import datetime

In [ ]:
now = datetime.now()

In [ ]:
players_df['Age'] = (now - players_df['BirthDate']).astype('<m8[Y]')

In [ ]:
# Who are the oldest players?
players_df.sort_values('Age', ascending=False).head()

In [ ]:
# Who are the youngest players?
players_df.sort_values('Age').head()

In [ ]:
avg_country_ages = players_df.groupby('TeamName')['Age'].mean().sort_values()

In [ ]:
teams_df["AverageAge"] = teams_df['TeamName'].apply(avg_country_ages.get)

### None of the teams with average age above 28 could make it past Quarterfinals... 

In [ ]:
teams_df.sort_values('AverageAge')[['TeamName', 'AverageAge', 'BestStage']]

## Some match attendance analysis...
![](https://i.imgur.com/MyTtkVy.jpg)

In [ ]:
# Attendance in France matches
matches_df.query("HomeTeamName == 'France' or AwayTeamName == 'France'").plot.bar(x="Stage", y="Attendance", figsize=(15,7))

In [ ]:
def get_avg_team_attendance(team_name):
    return matches_df.query("HomeTeamName == '{0}' or AwayTeamName == '{0}'".format(team_name))["Attendance"].mean()

In [ ]:
teams_df['AvgAttendance'] = teams_df['TeamName'].apply(get_avg_team_attendance)

In [ ]:
teams_df[['TeamName', 'AvgAttendance', 'BestStage']].sort_values('AvgAttendance', ascending=False)

In [ ]:
# Stage wise average attendance
matches_df.groupby("Stage")['Attendance'].mean().sort_values(ascending=False)

In [ ]:
matches_df.groupby("Stage")['Attendance'].mean().sort_values(ascending=False).plot.bar()

### Why so low attendance in quarters? Maybe bcoz of no Portugal, Spain and Argentina!

## Some analysis on team tactics...
![](https://theresonlyoneball.files.wordpress.com/2018/01/tactics.png?w=648)

In [ ]:
matches_df["HomeTeamTactics"].value_counts()

### 4-2-3-1 seems to be everyone's favorite. Does it guarantee a win?

In [ ]:
def set_winning_team_tactics(match):
    if match["Winner"] == match["HomeTeamName"]:
        return match["HomeTeamTactics"]
    elif match["Winner"] == match["AwayTeamName"]:
        return match["AwayTeamTactics"]
    else:
        return None

In [ ]:
matches_df["WinningTeamTactics"] = matches_df.apply(set_winning_team_tactics, axis=1)

In [ ]:
matches_df["WinningTeamTactics"].value_counts().plot.pie(autopct='%1.1f%%')

### 4-2-3-1 is used by the winning team maximum number of times...what about losing teams' tactics?

In [ ]:
def set_losing_team_tactics(match):
    if match["Winner"] == match["HomeTeamName"]:
        return match["AwayTeamTactics"]
    elif match["Winner"] == match["AwayTeamName"]:
        return match["HomeTeamTactics"]
    else:
        return None

In [ ]:
matches_df["LosingTeamTactics"] = matches_df.apply(set_losing_team_tactics, axis=1)

In [ ]:
matches_df["LosingTeamTactics"].value_counts().plot.pie(autopct='%1.1f%%')

### How does 4-2-3-1 tactic fares at big stage?

In [ ]:
matches_df.query("WinningTeamTactics == '4-2-3-1' and Stage != 'First stage'")

![](https://img.memecdn.com/leaked-portugal-world-cup-tactics_o_3411817.jpg)

## Some analysis on ball possession...

![](https://sportzwiki.com/wp-content/uploads/2018/03/sprinkle.jpg)

In [ ]:
def set_winning_team_ball_possession(match):
    if match["Winner"] == match["HomeTeamName"]:
        return match["BallPossessionHome"]
    elif match["Winner"] == match["AwayTeamName"]:
        return match["BallPossessionAway"]
    else:
        return None
    
def set_losing_team_ball_possession(match):
    if match["Winner"] == match["HomeTeamName"]:
        return match["BallPossessionAway"]
    elif match["Winner"] == match["AwayTeamName"]:
        return match["BallPossessionHome"]
    else:
        return None

In [ ]:
matches_df["BallPossessionWinner"] = matches_df.apply(set_winning_team_ball_possession, axis=1)
matches_df["BallPossessionLoser"] = matches_df.apply(set_losing_team_ball_possession, axis=1)

In [ ]:
matches_df.query("BallPossessionWinner < BallPossessionLoser").describe(include='object')

### In 4/6 matches, France had lesser ball possession than the opponent..

In [ ]:
def get_avg_team_ball_possession(team_name):
    home_possessions = matches_df.query("HomeTeamName == '{0}'".format(team_name))["BallPossessionHome"]
    away_possessions = matches_df.query("AwayTeamName == '{0}'".format(team_name))["BallPossessionAway"]
    return (sum(home_possessions) + sum(away_possessions))/ (len(home_possessions) + len(away_possessions))

In [ ]:
teams_df["AverageBallPossession"] = teams_df["TeamName"].apply(get_avg_team_ball_possession)

In [ ]:
teams_df[["TeamName", "AverageBallPossession", "BestStage"]].sort_values(by="AverageBallPossession", ascending=False)

![](http://ift.tt/UeVgbp)